## Inputs
---

In [1]:
# reading data and working with arrays
import h5py, nrrd
import numpy as np

# data paths
data_path = '/groups/scicompsoft/home/fleishmang/rnr-exm-work/data/sample_pair.h5'
hdf5_file = h5py.File(data_path, 'r')
fix = hdf5_file['/fixed'][...]
mov = hdf5_file['/move'][...]

# assumed spacings (there are no provided spacings, this is a problem)
fix_spacing = np.array([1., 1., 1.])
mov_spacing = np.array([1., 1., 1.])

# # write some channels
# nrrd.write('./fix.nrrd', fix.transpose(2,1,0), compression_level=2)
# nrrd.write('./mov.nrrd', mov.transpose(2,1,0), compression_level=2)

## Alignment
---

### affine

In [2]:
%%time

# alignment functions
from bigstream.align import alignment_pipeline
from bigstream.transform import apply_transform

# define alignment steps
common_kwargs = {
    'alignment_spacing':4.0,
    'shrink_factors':(2,),
    'smooth_sigmas':(2.,),
    'optimizer_args':{
        'learningRate':0.25,
        'minStep':0.,
        'numberOfIterations':400,
    },
}

affine_kwargs = {
    'initial_condition':'CENTER',
}

steps = [('affine', {**common_kwargs, **affine_kwargs},),]

# align
affine = alignment_pipeline(
    fix, mov,
    fix_spacing, mov_spacing,
    steps,
)

# apply affine only
affine_aligned = apply_transform(
    fix, mov,
    fix_spacing, mov_spacing,
    transform_list=[affine,],
)

# write results
np.savetxt('affine.mat', affine)
nrrd.write('./affine.nrrd', affine_aligned.transpose(2,1,0), compression_level=2)

# load precomputed results
affine = np.loadtxt('./affine.mat')

LEVEL:  0  ITERATION:  0  METRIC:  -0.06210452243082495
LEVEL:  0  ITERATION:  1  METRIC:  -0.06274774389745771
LEVEL:  0  ITERATION:  2  METRIC:  -0.0636490790318707
LEVEL:  0  ITERATION:  3  METRIC:  -0.06434227187925197
LEVEL:  0  ITERATION:  4  METRIC:  -0.0648048087891305
LEVEL:  0  ITERATION:  5  METRIC:  -0.06502328127789331
LEVEL:  0  ITERATION:  6  METRIC:  -0.06523588267937006
LEVEL:  0  ITERATION:  7  METRIC:  -0.06561278422748422
LEVEL:  0  ITERATION:  8  METRIC:  -0.06601412788940372
LEVEL:  0  ITERATION:  9  METRIC:  -0.06660213829967804
LEVEL:  0  ITERATION:  10  METRIC:  -0.06752170991556725
LEVEL:  0  ITERATION:  11  METRIC:  -0.06868228162099496
LEVEL:  0  ITERATION:  12  METRIC:  -0.06989599248525798
LEVEL:  0  ITERATION:  13  METRIC:  -0.07099974411530936
LEVEL:  0  ITERATION:  14  METRIC:  -0.07189742609915618
LEVEL:  0  ITERATION:  15  METRIC:  -0.07262966256204519
LEVEL:  0  ITERATION:  16  METRIC:  -0.07344749512342472
LEVEL:  0  ITERATION:  17  METRIC:  -0.0742